In [ ]:
# default_exp utils

# utils

> Supplies basic utility functions.

In [ ]:
#export
import datetime

import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, TableColumn, DataTable, LinearColorMapper, ColorBar
from bokeh.palettes import Viridis
import panel as pn
import panel.widgets as pnw
import pandas as pd

from icevision.visualize.draw_data import draw_record, draw_pred
from icevision.core.class_map import ClassMap

In [ ]:
pn.extension()

## Test data setup

In [ ]:
import icedata

In [ ]:
test_data_dir = icedata.fridge.load_data()
test_class_map = icedata.fridge.class_map()
test_parser = icedata.fridge.parser(test_data_dir, test_class_map)
test_train_records, test_valid_records = test_parser.parse()

## Data

In [ ]:
#export
def aggregate_record_data(records, class_map=None):
    """Aggregates stats from a list of records and returns a pandas dataframe with the aggregated stats. The creation time is not necessarily the real creation time. 
    This depends on the OS, for more information see: https://docs.python.org/3/library/os.html#os.stat_result."""
    data = []
    for index,record in enumerate(records):
        for label, bbox in zip(record["labels"], record["bboxes"]):
            file_stats = record["filepath"].stat()
            bbox_widht = bbox.xmax - bbox.xmin
            bbox_height = bbox.ymax - bbox.ymin
            area = bbox_widht * bbox_height
            area_normalized = area / (record["width"] * record["height"])
            bbox_ratio = bbox_widht / bbox_height
            data.append(
                {
                    "id": record["imageid"], "width": record["width"], "height": record["height"], "label": label, "bbox_xmin": bbox.xmin, 
                    "bbox_xmax": bbox.xmax, "bbox_ymin": bbox.ymin, "bbox_ymax": bbox.ymax, "area": area, "area_normalized": area_normalized,
                    "bbox_ratio": bbox_ratio, "record_index": index, "bbox_width": bbox_widht, "bbox_height": bbox_height, "filepath": str(record["filepath"]),
                    "creation_date": datetime.datetime.fromtimestamp(file_stats.st_ctime), "modification_date": datetime.datetime.fromtimestamp(file_stats.st_mtime),
                    "num_annotations": len(record["bboxes"])
                }
            )
    data = pd.DataFrame(data)
    data["label_num"] = data["label"]
    if class_map is not None:
        data["label"] = data["label"].apply(class_map.get_id)
    return data

In [ ]:
test_agg_data = aggregate_record_data(test_valid_records, test_class_map)
test_agg_data.head()

In [ ]:
#export
def calculate_dataset_stats(data):
    stats_dict = {}
    stats_dict["no_imgs"] = data["filepath"].nunique()
    stats_dict["no_classes"] = data["label"].nunique()
    stats_dict["classes"] = list(data["label"].unique())
    stats_dict["area_min"] = data["area"].min()
    stats_dict["area_max"] = data["area"].max()
    stats_dict["num_annotations_min"] = data["num_annotations"].min()
    stats_dict["num_annotations_max"] = data["num_annotations"].max()
    return stats_dict

In [ ]:
calculate_dataset_stats(test_agg_data)

In [ ]:
#export
def calculate_class_stats(data: pd.DataFrame):
    """Creates a dataframe containing stats about the object classes."""
    stats_dict = {}
    label_group = data.groupby("label")
    for label, group in label_group:
        label_stats = {}
        label_stats["imgs"] = group["filepath"].nunique()
        label_stats["objects"] = group.shape[0]
        label_stats["objects_per_img"] = label_stats["objects"]/label_stats["imgs"]
        label_stats["frac_of_labels"] = round(label_stats["objects"]/data.shape[0], 2)
        stats_dict[label] = label_stats
    df = pd.DataFrame(stats_dict).T
    df = df.rename_axis('Class').reset_index()
    return df

In [ ]:
calculate_class_stats(test_agg_data)

In [ ]:
#export
def calculate_image_stats(data: pd.DataFrame):
    """Creates a dataframe containing stats about the images."""
    stats_dict = {}
    stats_dict["Num. imgs."] = data["filepath"].nunique()
    stats_dict["Min Num. Objects"] = data["num_annotations"].min()
    stats_dict["Max Num. Objects"] = data["num_annotations"].max()
    stats_dict["Avg. Objects/Img"] = round(data["num_annotations"].mean(),2)
    df = pd.DataFrame.from_dict(stats_dict, orient="index").T
    return df

In [ ]:
calculate_image_stats(test_agg_data)

In [ ]:
#export
def calculate_mixing_matrix(data, mixing_col, mixing_objects, return_df=True):
    """Calculates mixing matrix for the mixing_objects column where they mix in the mixing_col. 
    By standard the object class mixing matrix over the images is calculated. 
    Returns the mixing matrix and the mapping between label and mixing matrix index.
    If return_df is True (default) a dataframe (instead of the mixing matrix) will be returned that can be directly consumed by histogram_2d."""
    # map labels to the mixing matrix index
    mapping = {i:j for j,i in enumerate(np.sort(data[mixing_objects].unique()))}
    mixing_matrix = np.zeros([data[mixing_objects].nunique(), data[mixing_objects].nunique()])
    mixing_groups = data.groupby(mixing_col)
    # iterate over each individual element with the same mixing_col to calculate the mixing based on the mixing_objects
    for group_key, group in mixing_groups:
        # handel self mixing
        for value, count in group[mixing_objects].value_counts().iteritems():
            if count > 1:
                mixing_matrix[mapping[value]] += 1
        # handel mixing of different objects
        permutations = np.array(np.meshgrid(group[mixing_objects].unique(), group[mixing_objects].unique())).T.reshape(-1,2)
        for permutation in permutations:
            # avoid double counting in the self mixing 
            if permutation[0] != permutation[1]:
                mixing_matrix[mapping[permutation[0]], mapping[permutation[1]]] += 1
    if return_df:
        df_dict = {"values": [], "col_name": [], "row_name": []}
        for row_name, row in zip(mapping, mixing_matrix):
            df_dict["values"] += row.tolist()
            df_dict["row_name"] += [row_name]*len(mapping)
            df_dict["col_name"] += mapping
        return pd.DataFrame(df_dict), mapping
    return mixing_matrix, mapping

Only counts once if objects occure in the same image not multiplt times.

In [ ]:
# test without class_map
test_mixing_matrix_df, test_mixing_matrix_mapping = calculate_mixing_matrix(aggregate_record_data(test_valid_records), mixing_col="filepath", mixing_objects="label")
# test class_map
test_mixing_matrix_df, test_mixing_matrix_mapping = calculate_mixing_matrix(test_agg_data, mixing_col="filepath", mixing_objects="label")

In [ ]:
test_mixing_matrix_df.head()

## Plotting

In [ ]:
#export
def convert_rgb_image_to_bokeh_rgb_image(img: np.ndarray):
    """Convertes a image in the form of a numpy array to an array that can be shown by bokeh."""
    img = np.flipud(img)
    img = img.astype(np.uint8)
    bokeh_img = np.empty((img.shape[0],img.shape[1]), dtype=np.uint32)
    view = bokeh_img.view(dtype=np.uint8).reshape((img.shape[0],img.shape[1], 4))
    view[:,:, 0] = img[:,:,0]
    view[:,:, 1] = img[:,:,1]
    view[:,:, 2] = img[:,:,2]
    view[:,:, 3] = 255
    return bokeh_img

Bokeh requries images to be in a hw format where each value is a 32bit integer where each of the 8bit sequences contains the rgb and alpha values.

In [ ]:
#hide
img = np.random.randint(0, 256, [10,10,3], dtype=np.uint8)
bokeh_img = convert_rgb_image_to_bokeh_rgb_image(img)
assert bokeh_img.shape == (10,10)
assert bokeh_img.dtype == np.uint32

In [ ]:
#export
def draw_record_with_bokeh(
    record,
    class_map=None,
    display_label=True,
    display_bbox=False,
    display_mask=False,
    display_keypoints=False,
    return_figure=False,
    width=None,
    height=None
):
    """Draws a record or returns a bokeh figure containing the image."""
    img = draw_record(
            record=record,
            class_map=class_map,
            display_label=display_label,
            display_bbox=display_bbox,
            display_mask=display_mask,
            display_keypoints=display_keypoints,
        )

    # create bokeh figure with the plot
    bokeh_img = convert_rgb_image_to_bokeh_rgb_image(img)
    
    # make sure the aspect ratio of the image is retained, if only the width of hight is given
    if width is None and height is not None:
        plot_width = int(img.shape[1]/img.shape[0] * height)
        plot_height = height
    elif height is None and width is not None:
        plot_width = width
        plot_height = int(img.shape[0]/img.shape[1] * width)
    else:
        plot_width = img.shape[1] if width is None else width
        plot_height = img.shape[0] if height is None else height
    
    p = figure(tools="reset, wheel_zoom, box_zoom, save, pan", width=plot_width, height=plot_height, x_range=(0, img.shape[1]), y_range=(img.shape[0], 0), x_axis_location="above")
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.image_rgba([bokeh_img], x=0, y=img.shape[0], dw=img.shape[1], dh=img.shape[0], level="image")
    if return_figure:
        return p
    else:
        show(p)

Wrapper around the `draw_record` function from icevision. The aspect ratio of the image will be preserved when only width or height is given (scaling the other accordingly).

In [ ]:
draw_record_with_bokeh(test_train_records[0], width=200)

In [ ]:
#export
def barplot(counts, values, class_map=None, bar_type="horizontal", width=500, height=500):
    """Creates a figure with a barplot, were the counts is the bar height and values are the labels for the bars."""
    if class_map is None:
        values = [str(entry) for entry in values]
    else:
        values = [class_map.get_id(entry) for entry in values]
    if bar_type == "horizontal":
        p = figure(width=width, height=height, y_range=values)
        p.hbar(y=values, left=0, right=counts, height=0.9)
    elif bar_type == "vertical":
        p = figure(width=width, height=height, x_range=values)
        p.vbar(x=values, bottom=0, top=counts, width=0.9)
    else:
        raise ValueError("hist_type has to be of 'horizontal' or 'vertical'")
    return p

In [ ]:
# test draw_histogram without a classmap
p = barplot([10, 20], [1,2])
pn.Row(p)

In [ ]:
# test draw_histogram without a classmap
hist = [[1, 10], [2, 20]]
cls_map = ClassMap(["a", "b"])
p = barplot([10, 20], [1, 2], cls_map, bar_type="vertical")
pn.Row(p)

In [ ]:
#export
def histogram(values, bins=10, range=None, density=False, plot_figure=None, remove_tools=False, width=500, height=500):
    "Creates a histogram"
    if plot_figure is None:
        p = figure(width=width, height=height)
    else:
        p = plot_figure
    counts, edges = np.histogram(values, bins=bins, range=range, density=density)
    p.quad(top=counts, bottom=0, left=edges[:-1], right=edges[1:])
    if remove_tools:
        p.toolbar.logo = None
        p.toolbar_location = None
    return p

In [ ]:
pn.Row(histogram([1,1,1,1,2,2,2,3,3,4], bins=4, range=(1,4), density=True))

In [ ]:
p = figure(title="Test")
p = histogram([1,1,1,1,2,2,2,3,3,4], bins=4, plot_figure=p)
pn.Row(p)

In [ ]:
#export
def histogram_2d(df, x, y, values, color_mapper=None, height=500, width=500):
    if color_mapper is None:
        color_mapper = LinearColorMapper(palette=Viridis[256], low=df[values].min(), high=df[values].max())
    
    # ensure the x and y column are in a categorical format
    if df[x].dtype != str or df[y].dtype != str:
        df = df.copy()
        df[x] = df[x].astype(str)
        df[y] = df[y].astype(str)
    
    p = figure(
        title="Classes in image mixing matrix", x_range=sorted(df[x].unique())[::-1], y_range=sorted(df[y].unique()), 
        x_axis_location="above", tools="hover", toolbar_location=None, tooltips=[('Mixing images: ', '@'+values)],
        width=width, height=height
    )

    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_standoff = 0

    p.rect(
        x=x, y=y, width=1, height=1, source=df,
        fill_color={'field': values, 'transform': color_mapper},
        line_color=None
    )

    color_bar = ColorBar(
        color_mapper=color_mapper, major_label_text_font_size="7px",
        #ticker=BasicTicker(desired_num_ticks=len(colors)),
        label_standoff=6, border_line_color=None, location=(0, 0)
    )
    p.add_layout(color_bar, 'right')

    return p

In [ ]:
pn.Column(histogram_2d(test_mixing_matrix_df, "row_name", "col_name", "values"))

In [ ]:
#export
def time_arc_plot(start_date, end_date, plot_figure=None, width=500, height=300):
    radius = (end_date-start_date)/2
    x = start_date + radius
    if plot_figure is None:
        p = figure(x_axis_label="Date", x_axis_type='datetime', y_range=(0, radius.max()), x_range=(start_date.min(), end_date.max()), width=width, height=height)
        p.yaxis.major_tick_line_color = None
        p.yaxis.minor_tick_line_color = None
        p.yaxis.major_label_text_font_size = '0pt'
        p.toolbar.logo = None
        p.toolbar_location = None
    else:
        p = plot_figure
    p.arc(x=x, y=0, radius=radius, start_angle=0, end_angle=np.pi)
    return p

In [ ]:
random_time_delta = pd.DataFrame([datetime.timedelta(days=np.random.randint(0, 50)) for _ in range(test_agg_data.shape[0])], columns=["t_delta"])
pn.Row(time_arc_plot(test_agg_data["creation_date"], test_agg_data["modification_date"]+random_time_delta["t_delta"]))

In [ ]:
#export
def table_from_dataframe(dataframe, columns=None, width=500, height=200, index_position=None):
    if columns is not None:
        selection = dataframe[columns]
    else:
        selection = dataframe
    source = ColumnDataSource(selection)
    table_cols = [TableColumn(field=filed, title=filed.replace("_", " ").title()) for filed in selection.columns]
    data_table = DataTable(source=source, columns=table_cols, width=width, height=height)
    data_table.index_position = index_position
    return data_table

In [ ]:
pn.Row(table_from_dataframe(test_agg_data, width=1000))

## Controll elements

In [ ]:
#export
def generate_range_filter(data, name, with_hist=True, steps=50, height=500, width=500):
    "Generates a range slider with a histogram (if with_hist is True) for a given pd.DataFrame and a column key."
    val_min = data.min()
    val_max = data.max()
    # subtract and add a bit to the min and max value to ensure the whole range is captured
    dist = val_max-val_min if val_max != val_min else 1
    val_min = val_min-0.01*dist
    val_max = val_max+0.01*dist
    slider = pnw.RangeSlider(name=name, start=val_min, end=val_max, step=round(((val_max-val_min)/steps), 1), width=width)
    if with_hist:
        hist = histogram(data, bins=20, height=100, width=width, remove_tools=True)
    else:
        hist = None
    range_filter = pn.Column(slider, hist, "<br>")
    return range_filter

In [ ]:
generate_range_filter(test_agg_data["area"], "Area", width=600)

In [ ]:
#export
def get_min_and_max_dates(dates):
    min_date = dates.min().to_pydatetime().replace(microsecond=0, second=0, minute=0, hour=0)
    max_date = dates.max().to_pydatetime().replace(microsecond=0, second=0, minute=0, hour=0)
    # make sure the min and max values are at least a day appart
    if min_date == max_date:
        max_date = max_date.replace(day=max_date.day+1)
    return min_date, max_date

In [ ]:
#export
def generate_creation_modification_time_filter(data, width=500, height=500):
    """Generates an arc plot with creation and modification time and two range sliders to select parts for the two."""
    plot = time_arc_plot(data["creation_date"], data["modification_date"], width=width)
    min_creation_date, max_creation_date = get_min_and_max_dates(data["creation_date"])
    min_modification_date, max_modification_date = get_min_and_max_dates(data["modification_date"])
    min_date = min(min_creation_date, min_modification_date)
    max_date = max(max_creation_date, max_modification_date)
    creation_time_slider = pnw.DateRangeSlider(name="Creation Time", start=min_date, end=max_date, width=width)
    modification_time_slider = pnw.DateRangeSlider(name="Modification Time", start=min_date, end=max_date, width=width)
    return pn.Column(plot, creation_time_slider, modification_time_slider)

In [ ]:
test_time_selection = test_agg_data.copy()
test_time_selection["modification_date"] += random_time_delta["t_delta"]
generate_creation_modification_time_filter(test_time_selection, width=200)